# Name : Shreya Sadashiva Kamath

# Git-Hub Username : shreyasa

# USC ID: 7553148882 

In [1]:
# Import statements
import pandas as pd
import glob
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score
import math
from tabulate import tabulate
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
import glob
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import numpy as np
import scipy.stats as st
from pathlib import Path
import os
import csv
import warnings
from io import StringIO

In [2]:
dataPath = './Data/AReM/ARem/'

## 1. b) Creating Training and Tetsing Data

In [4]:
dir1 = ['bending1', 'bending2']
dir2 = ['cycling', 'lying', 'sitting', 'standing', 'walking']
train, test, allCsvs = [], [], []
curr=[]

for d in dir1:
    for filename in os.listdir(dataPath+d):
        curr.append(dataPath+d+"/"+filename)
        if filename[-6:-4] in ('t1', 't2'):
            test.append(dataPath+d+"/"+filename)
        else:
            train.append(dataPath+d+"/"+filename)
for d in dir2:
    for filename in os.listdir(dataPath+d):
        curr.append(dataPath+d+"/"+filename)
        if filename[-6:-4] in ('t1', 't2' , 't3'):
            test.append(dataPath+d+"/"+filename)
        else:
            train.append(dataPath+d+"/"+filename)
curr.sort()
allCsvs.append(curr)

In [5]:
test

['./AReM/ARem/bending1/dataset1.csv',
 './AReM/ARem/bending1/dataset2.csv',
 './AReM/ARem/bending2/dataset1.csv',
 './AReM/ARem/bending2/dataset2.csv',
 './AReM/ARem/cycling/dataset1.csv',
 './AReM/ARem/cycling/dataset2.csv',
 './AReM/ARem/cycling/dataset3.csv',
 './AReM/ARem/lying/dataset1.csv',
 './AReM/ARem/lying/dataset2.csv',
 './AReM/ARem/lying/dataset3.csv',
 './AReM/ARem/sitting/dataset1.csv',
 './AReM/ARem/sitting/dataset2.csv',
 './AReM/ARem/sitting/dataset3.csv',
 './AReM/ARem/standing/dataset1.csv',
 './AReM/ARem/standing/dataset2.csv',
 './AReM/ARem/standing/dataset3.csv',
 './AReM/ARem/walking/dataset1.csv',
 './AReM/ARem/walking/dataset2.csv',
 './AReM/ARem/walking/dataset3.csv']

In [6]:
 train

['./AReM/ARem/bending1/dataset3.csv',
 './AReM/ARem/bending1/dataset4.csv',
 './AReM/ARem/bending1/dataset5.csv',
 './AReM/ARem/bending1/dataset6.csv',
 './AReM/ARem/bending1/dataset7.csv',
 './AReM/ARem/bending2/dataset3.csv',
 './AReM/ARem/bending2/dataset4.csv',
 './AReM/ARem/bending2/dataset5.csv',
 './AReM/ARem/bending2/dataset6.csv',
 './AReM/ARem/cycling/dataset10.csv',
 './AReM/ARem/cycling/dataset11.csv',
 './AReM/ARem/cycling/dataset12.csv',
 './AReM/ARem/cycling/dataset13.csv',
 './AReM/ARem/cycling/dataset14.csv',
 './AReM/ARem/cycling/dataset15.csv',
 './AReM/ARem/cycling/dataset4.csv',
 './AReM/ARem/cycling/dataset5.csv',
 './AReM/ARem/cycling/dataset6.csv',
 './AReM/ARem/cycling/dataset7.csv',
 './AReM/ARem/cycling/dataset8.csv',
 './AReM/ARem/cycling/dataset9.csv',
 './AReM/ARem/lying/dataset10.csv',
 './AReM/ARem/lying/dataset11.csv',
 './AReM/ARem/lying/dataset12.csv',
 './AReM/ARem/lying/dataset13.csv',
 './AReM/ARem/lying/dataset14.csv',
 './AReM/ARem/lying/dataset1

## c) Feature Extraction

### i) Research: Time-domain features used in time series classification: 

**Mean(Average)**: Represents the central tendency of the data.

**Minimum**: It is the minimum value of the dataset.

**Maximum**: Largest value in the time series.

**Median**: It is the middle value which divides the upper half and lower half of the dataset.

**Standard Deviation**: Measures the spread or dispersion of the data points from the mean.

**1st Quartile**: It the value under which 25% if the data points are found in a dataset.

**3rd Quartile**: It the value under which 75% if the data points are found in a dataset.

**Interquartile Range**: It is the range between 25% and 75%, where the middle 50% of the data points lie.

**Range**: It is the difference between the min and max values of a dataset.

**Variance**: It is mean squared difference between each data point and the centre of the distribution measured by the mean.

**skewness**: Measures the asymmetry of the data distribution.

**Kurtosis**: Measures the "tailedness" or the heaviness of the tails in the data distribution.

**Zero Crossing Rate**: Counts the number of times the signal changes sign.

**Entropy**: Quantifies the amount of information or disorder in the data.

**Autocorrelation**: Measures the correlation between a time series and a lagged version of itself.

**Cross-Correlation**: Measures the correlation between two different time series.

**Energy**: Represents the total energy contained in the time series.

### (ii) Time-domain features for the data 

In [7]:
COLUMN_NAMES = [ "min1", "max1", "mean1", "median1", "std1", "1st quartile1", "3rd quartile1",
                 "min2", "max2", "mean2", "median2", "std2", "1st quartile2", "3rd quartile2",
                 "min3", "max3", "mean3", "median3", "std3", "1st quartile3", "3rd quartile3",
                 "min4", "max4", "mean4", "median4", "std4", "1st quartile4", "3rd quartile4",
                 "min5", "max5", "mean5", "median5", "std5", "1st quartile5", "3rd quartile5",
                 "min6", "max6", "mean6", "median6", "std6", "1st quartile6", "3rd quartile6"]

def get_delimiter(file):
    sniffer = csv.Sniffer()
    f = open(file)
    return sniffer.sniff(f.read()).delimiter


# Define function to load data from a folder
def load_data_from_folder(folder_path):
    data = pd.DataFrame()

    try:
      delimiter = get_delimiter(folder_path)
      df = []
      if(delimiter == ','):
        df = pd.read_csv(folder_path, delimiter=',', header=None, skiprows=5)
      elif(delimiter==' '):
        df = pd.read_csv(folder_path, delimiter='\\s+', header=None, skiprows=5)
      else:
        df = pd.read_csv(folder_path, header=None, skiprows=5, on_bad_lines='skip')

      df.drop(columns=df.columns[0], axis=1, inplace=True) #drop the 'time' column
      data = pd.concat([data, df], axis=1)


    except Exception as e:
            # some files have one trailing comma at the end

            # Read the entire CSV file as text
            with open(folder_path, 'r') as input_file:
                csv_content = input_file.read()

            # Check if the CSV content ends with a comma
            if csv_content.endswith(','):
                # Remove the trailing comma from the end of the content
                csv_content = csv_content[:-1]  # Remove the last character (comma)

            # Use StringIO to read the modified CSV content as a DataFrame
            df = pd.read_csv(StringIO(csv_content),delimiter=',', header=None,skiprows=5)

            df.drop(columns=df.columns[0], axis=1, inplace=True) #drop the 'time' column
            data = pd.concat([data, df], axis=1)

    return data


def extract_time_series_features(data):
  time_series_features = pd.DataFrame()
  for folder in data:
      folder_path = f'{folder}'
      df = load_data_from_folder(folder_path)
      # Calculate the describe statistics for each column
      describe_stats = df.describe().loc[['min', 'max', 'mean', '50%', 'std', '25%', '75%']].T

      # Flatten the describe statistics into a single row
      flatten_stats = describe_stats.values.flatten()

      final_df = pd.DataFrame([flatten_stats], columns=COLUMN_NAMES)

      #extract folder name from file path
      second_last_index = folder.rfind("/", 0, folder.rfind("/"))
      final_df['folder'] = folder[second_last_index+1:folder.rfind("/")]

      time_series_features = pd.concat([time_series_features,final_df])
  
  time_series_features = time_series_features.reset_index(drop=True)
  return time_series_features

In [8]:
#Time-domain Features for Train Data
train_features = pd.DataFrame()
train_features = extract_time_series_features(train)
display(train_features)

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,1st quartile5,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder
0,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400,bending1
1,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000,bending1
2,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000,bending1
3,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,22.2500,24.00,0.0,5.26,0.679646,0.500,0.622534,0.43,0.8700,bending1
4,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,20.5000,23.75,0.0,2.96,0.555312,0.490,0.487826,0.00,0.8300,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050,walking
65,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000,walking
66,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200,walking
67,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,4.398833,...,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600,walking


In [9]:
#Time-domain Features for Test Data
test_features = extract_time_series_features(test)
display(test_features)

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,1st quartile5,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
3,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
4,24.25,45.00,37.177042,36.250,3.581301,34.5000,40.2500,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.900,cycling
5,28.75,44.75,37.561187,36.875,3.226507,35.2500,40.2500,0.0,9.91,2.080687,...,18.0000,21.5000,0.0,9.62,2.765896,2.450,1.769203,1.4100,3.770,cycling
6,22.00,44.67,37.058708,36.000,3.710180,34.5000,40.0625,0.0,14.17,2.438146,...,16.0000,21.0000,0.0,8.55,2.983750,2.570,1.815730,1.5000,4.150,cycling
7,23.50,30.00,27.716375,27.500,1.442253,27.0000,29.0000,0.0,1.79,0.363687,...,5.5000,10.7500,0.0,4.50,0.734271,0.710,0.613688,0.4300,1.000,lying
8,24.75,48.33,44.182937,48.000,7.495615,48.0000,48.0000,0.0,3.11,0.101875,...,2.0000,5.5425,0.0,3.91,0.692771,0.500,0.675781,0.3225,0.940,lying
9,48.00,48.25,48.004167,48.000,0.032038,48.0000,48.0000,0.0,0.43,0.007167,...,4.6700,10.0000,0.0,2.50,0.641229,0.500,0.388372,0.4600,0.830,lying


In [10]:
#New Dataset with total train and test features
train_features['label'] = 0
test_features['label'] = 1
time_series_dataset = pd.concat([train_features, test_features])
display(time_series_dataset)

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder,label
0,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,36.500,0.0,1.79,0.493292,0.430,0.513506,0.000,0.940,bending1,0
1,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,36.330,0.0,2.18,0.613521,0.500,0.524317,0.000,1.000,bending1,0
2,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,31.250,0.0,1.79,0.383292,0.430,0.389164,0.000,0.500,bending1,0
3,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,24.000,0.0,5.26,0.679646,0.500,0.622534,0.430,0.870,bending1,0
4,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,23.750,0.0,2.96,0.555312,0.490,0.487826,0.000,0.830,bending1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,35.50,46.25,43.174938,43.670,1.989052,42.50,44.5000,0.0,2.12,0.506583,...,16.500,0.0,5.72,0.911979,0.830,0.666161,0.470,1.220,standing,1
15,32.75,47.00,42.760562,44.500,3.398919,41.33,45.3725,0.0,3.34,0.486167,...,18.565,0.0,5.73,0.842271,0.710,0.722165,0.430,1.090,standing,1
16,19.33,43.50,34.227771,35.500,4.889576,30.50,37.7500,0.0,14.50,3.995729,...,18.670,0.0,9.74,3.394125,3.100,1.792090,2.105,4.425,walking,1
17,12.50,45.00,33.509729,34.125,4.850923,30.50,36.7500,0.0,13.05,4.450771,...,18.750,0.0,8.96,3.378479,3.085,1.787360,2.060,4.440,walking,1


### iii) Standard Deviation

In [13]:
def calculate_std_ci(dataset):
  std_dev_and_ci = {}

  for feature in dataset.columns:
    if feature not in ['folder','label']:
      data = dataset[feature].values
      # Calculate the standard deviation of the original data
      std_dev = np.std(data)

      # Perform bootstrap resampling to create confidence intervals
      resamples = bs.bootstrap(data, stat_func=bs_stats.std, num_iterations=10000, num_threads=1)

      # Extract the 90% confidence interval
      lower_bound = resamples.lower_bound
      upper_bound = resamples.upper_bound

      std_dev_and_ci[feature] = {
          'Standard Deviation': std_dev,
          '90% CI Lower Bound': lower_bound,
          '90% CI Upper Bound': upper_bound
      }

  table = [['Feature', 'Standard Deviation','90% CI Lower Bound','90% CI Upper Bound']]
  for feature, values in std_dev_and_ci.items():
    table.append([feature,values['Standard Deviation'], values['90% CI Lower Bound'], values['90% CI Upper Bound']])


  print(tabulate(table, headers='firstrow', tablefmt='grid'))

In [15]:
calculate_std_ci(time_series_dataset)

+---------------+----------------------+----------------------+----------------------+
| Feature       |   Standard Deviation |   90% CI Lower Bound |   90% CI Upper Bound |
+===============+======================+======================+======================+
| min1          |             9.51545  |             8.04974  |           11.0431    |
+---------------+----------------------+----------------------+----------------------+
| max1          |             4.36932  |             3.30131  |            5.61727   |
+---------------+----------------------+----------------------+----------------------+
| mean1         |             5.30531  |             4.64131  |            6.05606   |
+---------------+----------------------+----------------------+----------------------+
| median1       |             5.40906  |             4.71886  |            6.15897   |
+---------------+----------------------+----------------------+----------------------+
| std1          |             1.76206  |   

### iv) Three most important time-domain features

According to me the three most important time-domain features are:

**Mean**: Because it contains the score of each observation in the dataset and is good measure of the central tendency of the data when distribution is continuous.

**Median**: It provides us the central value of the dataset and gives us an idea of the central tendency of the dataset and is an important metric when we have outliers and skewed data in the dataset.

**Standard Deviation**: It gives us an ideal measure of dispersion of the data.


## 2. ISLR 3.7.4

a) Although, the true relationship between X and Y is linear the 'cubic' regression model might fit the training the training data really well as it has more flexibility and hence, the training RSS might be 'lower' for the cubic regression model compared to the linear one.

b) As we do not have enough information about the testing data, we cannot conclude anything perfectly. However, since the true relationship between X and Y is linear it will have lower RSS as compared to the cubic regression model as it will overfit the data and would result is higher RSS error.

(c) The train RSS for the cubic regression model might be lower than the train RSS for the linear model as the true relationship is non-linear, the cubic regression will fit better than the linear regression model.

(d) On Test data, it depends on the underlying relationship whether the linear regression would do better or the cubic regression. Hence, there is not enough information to a concrete answer. However, if the true relationship between X and Y is not very far from linear, the linear regression RSS would be lower. If the true relationship is very far from linear, the cubic regression RSS would be lower.

## 3. ISLR 3.7.3 - Extra Practice
## 3. ISLR 3.7.5 - Extra Practice